In [1]:
import gettext
import math
import random
import sys ## sleep 같은 것을 줄때
from time import sleep
import pygame
from pygame.locals import * 

##### 프로그램 전체 초기화 및 변수 설정###

pygame.init() ######### pygame 초기화

# 배경이미지의 크기가 1000 X 800 임
Window_Width = 1000
Window_Height = 600

BLACK = (0,0,0)
BB = (10,22,80)
WHITE = (250,250,250)
YELLOW = (250,250,20)
BLUE = (102,79,90)

## 게임 틀 설정 ##
screen = pygame.display.set_mode((Window_Width, Window_Height))
pygame.display.set_caption('대학생활 -현실편-') ## 게임 타이틀
pygame.display.set_icon(pygame.image.load('main_character.png')) ## 게임 아이콘
fps_clock = pygame.time.Clock() ## 게임 프레임
FPS = 60 ## 초당 60프레임


#### 주요 파일 불러오기/ 배경화면, 배경음, 효과음 ###
base_font = pygame.font.Font('Recipekorea_FONT.ttf', 30) ## 글폰트
background = pygame.image.load('background.jpg')  ## 게임 배경
background2 = pygame.image.load('result_screen.jpg')  ## 게임 배경
Item_sound = pygame.mixer.Sound('item2_sound.wav') ## 아이템획득효과음
bomb_sound = pygame.mixer.Sound('bomb_sound.wav') ## 과제폭탄 맞는 소리
bomb_effect = pygame.image.load('bomb2.png') ## 과제폭탄 맞을 때 효과 이미지
pygame.mixer.music.load('bensound-creativeminds.mp3') ## 배경음 
appearence_professor = pygame.mixer.Sound('Rainbow Disco Bears by edemson86 Id-322071.wav')# 교수님 등장
quiz_sound = pygame.mixer.Sound('item1_sound.wav') ## quiz아이템 획득 효과음
initial_image = pygame.image.load('대학생활 -현실편-2.jpg') ## 시작화면
initial_explain = pygame.image.load('initial_explain.jpg') ## 시작화면 2
F_sound = pygame.mixer.Sound('뿌직.wav') # F 먹을때 소리
girlfriend_sound = pygame.mixer.Sound('girlfriend_sound.wav')
lightening_sound = pygame.mixer.Sound('lightening_sound.wav')



back_rect = background.get_rect() 
# 교수님이 뿌리시는 학점
score_A = pygame.image.load('A.png')
score_B = pygame.image.load('B.png')
score_C = pygame.image.load('C.png')
score_D = pygame.image.load('D.png')
score_F = pygame.image.load('F.png')

# 결과창에 띄울 점수들
Academic_score = 0
Social_score = 0
Achievements=[] # 획득한 업적
Bonus = 0 # 업적보너스점수



######################################################################
class Main_character(pygame.sprite.Sprite):
    def __init__(self, xpos ,ypos):
        super(Main_character, self).__init__() # 부모의 초기화를 오버라이딩한다.
        self.image = pygame.image.load('main_character.png')
        self.rect = self.image.get_rect()
        self.rect.x = xpos
        self.rect.y = ypos
        self.centerx = self.rect.centerx
        self.centery = self.rect.centery
        # 주인공 이미지의 가로와 세로 길이
        self.size = self.image.get_rect().size
        self.width = self.size[0]
        self.height = self.size[1]
        
        
    # 경계밖으로 넘어가면 넘어가지 않도록 조정
    def set_pos(self, x,y):  #위치조정
        self.rect.x = x 
        if self.rect.x < 0 :
            self.rect.x = 0
        elif self.rect.x > Window_Width - self.width:
            self.rect.x = Window_Width - self.width
        self.rect.y = y 
        if self.rect.y < 0 :
            self.rect.y = 0
        elif self.rect.y > Window_Height - self.height:
            self.rect.y = Window_Height - self.height
        
    def collide(self, sprites): # 충돌판정
        for sprite in sprites: # sprites에 포함된 모든 객체들을 검사
            if pygame.sprite.collide_rect(self, sprite): # pygame의 충돌판정함수사용
                return sprite # 충돌된 sprite를 반환
######################################################################

## 화면의 상단 좌 우측에서 Bomb이 생성된다. ##
class Bomb(pygame.sprite.Sprite):
    def __init__(self, xpos, ypos, hspeed, vspeed):
        super(Bomb, self).__init__()  ## 부모 오버라이딩
        bomb = pygame.image.load('bomb.png')
        self.image = bomb
        self.rect = self.image.get_rect()
        self.rect.x = xpos
        self.rect.y = ypos
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.set_direction()
    
    # 과제폭탄이 날아가는 방향에 따라 이미지의 기울기가 달라진다.
    def set_direction(self):
        if self.vspeed > 0:
            if self.hspeed > 0:
                self.image = pygame.transform.rotate(self.image, 305)
            elif self.hspeed < 0:
                self.image = pygame.transform.rotate(self.image, 45)
        else:
            if self.hspeed > 0:
                self.image = pygame.transform.rotate(self.image, 270)
            elif self.hspeed < 0:
                self.image = pygame.transform.rotate(self.image, 90)
    
   # 폭탄이 움직이는 것을 갱신해주는 함수
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵 바깥으로 나갈 때
        if self.collide():
            self.kill() # 객체가 사라짐
            
    #맵 바깥으로 나가면 객체 삭제(bool)
    def collide(self):
        if self.rect.x < 0 - self.rect.height or self.rect.x > Window_Width:
            return True
        elif self.rect.y < 0 - self.rect.height or self.rect.y > Window_Height:
            return True
    
    def explosion(self):
        bomb_sound.play()
        screen.blit(bomb_effect, [self.rect.x , self.rect.y])
        self.kill()
            
#######################################################################
# 실질적으로 Bomb을 생성하는 부분. 방향과 속력을 결정한다.(랜덤으로)
def random_bomb(speed):
    random_direction = random.randint(1,3) # screen을 기준으로 3방향
    RandD = random.randint(1,3) # 기준선을 기준으로 세방향의 기울기
    
    #폭탄이 떨어지는 방향을 설정한다. 상, 좌 , 우에서 출현한다.
    #Rock(xpos, ypos, hspeed, vspeed)
    #위에서 아래로
    if random_direction == 1:
        if RandD == 1:
            return Bomb(random.randint(0, Window_Width-30), 0, 0, speed)
        elif RandD == 2:
            return Bomb(random.randint(0, Window_Width-30), 0, speed, speed)
        elif RandD == 3:
            return Bomb(random.randint(0, Window_Width-30), 0, -speed, speed)
        
    # 오른쪽에서 왼쪽
    elif random_direction == 2:
        if RandD == 1:
            return Bomb(Window_Width, random.randint(0, Window_Height-30), -speed, 0)
        elif RandD == 2:
            return Bomb(Window_Width, random.randint(0, Window_Height-30), -speed, speed)
        elif RandD == 3:
            return Bomb(Window_Width, random.randint(0, Window_Height-30), -speed, -speed)
    #왼쪽에서 오른쪽
    elif random_direction == 3:
        if RandD == 1:
            return Bomb(0, random.randint(0, Window_Width-30), speed, 0)
        elif RandD == 2:
            return Bomb(0, random.randint(0, Window_Width-30), speed, speed)
        elif RandD == 3:
            return Bomb(0, random.randint(0, Window_Width-30), speed, -speed)

#######################################################################
# 속도와 좌표를 설정해주면 친구 캐릭터가 화면을 떠다니게 된다. 
# 생성되고 벽과 5번 출돌하면 사라진다.
class Friend(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(Friend, self).__init__()
        self.image = pygame.image.load('friend.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        
        ## 맵전체에서 랜덤생성
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 ##
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
            
            
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2
        
#######################################################################
# 속도와 좌표를 설정해주면 Quiz item이 화면을 떠다니게 된다. 
# 생성되고 벽과 5번 출돌하면 사라진다.
class Quiz(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(Quiz, self).__init__()
        self.image = pygame.image.load('quiz.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        
        ## 맵전체에서 랜덤생성
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
            
            
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2
############################################################################        
# 속도와 좌표를 설정해주면 '연인' item이 화면을 떠다니게 된다. 
# 생성되고 벽과 5번 출돌하면 사라진다.
class GirlFriend(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(GirlFriend, self).__init__()
        self.image = pygame.image.load('girlfriend.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        
        ## 맵전체에서 랜덤생성
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
            
            
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2

        
#######################################################################
# 속도와 좌표를 설정해주면 '결별' item이 화면을 떠다니게 된다. 
# 생성되고 벽과 5번 출돌하면 사라진다.
class exGF(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(exGF, self).__init__()
        self.image = pygame.image.load('lightning.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        
        ## 맵전체에서 랜덤생성
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
            
            
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2
##############################################################################
# A ~ F 객체 
class Score(pygame.sprite.Sprite):
    def __init__(self, x, y, hspeed, vspeed, score):
        super(Score, self).__init__()
        ## 스코어 선택
        if score == 0:
            self.image = pygame.image.load('A.png')
        elif score == 1:
            self.image = pygame.image.load('B.png')
        elif score == 2:
            self.image = pygame.image.load('C.png')
        elif score == 3:
            self.image = pygame.image.load('D.png')
        elif score == 4:
            self.image = pygame.image.load('F.png')
        
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.time = 0
        self.s = score # judge 하는데 필요 
        

        ## 생성위치
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 ##
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        self.time +=1
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
            
        # 등장시간 10초
        if self.time == 600:
            self.kill()
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            return 2
        
    # 점수 계산하여 반환/ 소리재생
    def judge_point(self):
        if self.s == 0: # A
            Item_sound.play()
            return 50
        elif self.s == 1 : # B
            Item_sound.play()
            return 40
        elif self.s == 2 : # C
            Item_sound.play()
            return 30
        elif self.s == 3 : # D
            Item_sound.play()
            return 10
        elif self.s == 4 : # F
            F_sound.play()
            return -20
        
###########################################################################
def spread_scores(scores, proX, proY):
    # 가로와 세로속도 랜덤, 퍼센트에 따라 A 부터 F까지 생성한다. 총 20개의 스코어 생성
    lst = []
    for i in range(1,21):
        lst.append(i)
    plma = [-1.5,-1,-0.5, 0.5, 1, 1.5] 
    
    for i in range(20):
        a = random.choice(lst)
        pm = random.choice(plma)
        hsp = random.randint(2,4) * pm # 왼쪽 오른쪽 모두 출발하도록 방향을 부여한다. 총12방향
        vsp = random.randint(2,4) # 아래방향으로만 출발하도록 기본속도인 3을 넣어준다.
        if a == 1 or a == 2 :
            score = Score(proX, proY ,hsp, vsp,0) # A
        elif a >= 3 and a <= 5:
            score = Score(proX, proY ,hsp, vsp,1) # B
        elif a >= 6 and a <= 9:
            score = Score(proX, proY ,hsp, vsp,2) # C
        elif a >= 10 and a <= 14:
            score = Score(proX, proY, hsp, vsp,3) # D
        elif a >= 15 and a <= 20:
            score = Score(proX, proY ,hsp, vsp,4) # F
        scores.add(score)    


############################################################################
# 총 3단계의 보스 교수님.. 시험을 보고 점수를 뿌리신다.
class Professor(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(Professor, self).__init__()
        self.image = pygame.image.load('professor.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.time = 0
        ## 생성위치
        self.rect.centerx = x
        self.rect.centery = y
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        self.time += 1
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 등장시간 10초
        if self.time == 600:
            self.kill()
             
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            return 2
        

                


#######################################################################
## 텍스트를 그리기 위한 함수
def draw_text(text, font, surface, x, y, color):
    text_obj = font.render(text, True, color)
    text_rect = text_obj.get_rect()
    text_rect.centerx = x
    text_rect.centery = y
    surface.blit(text_obj, text_rect)
#  surface.blit() 메소드는 다른 Surface 객체를 자신에게 그려넣는 일을 한다. 
# 이 메소드의 1번째 매개변수는 그려넣을 Surface 객체, 
# 2번째 매개변수에 해당 Surface 객체를 그려넣을 좌표

## 폰트설정 ##
# font = pygame.font.Font('LexiGulim.ttf',30)  #폰트 설정
# text = font.render("글자출력",True,(28,0,0))  #텍스트가 표시된 Surface 를 만듬 (쓸 텍스트, 안티에일리어싱사용여부, 글자색)
# background.blit(text,(870,20))              #화면에 표시 surface 객체(text)를 background에 겹쳐서 표시


#########################################################################

def Run_game():
    
    global Academic_score, Social_score, Bonus, Achievements
    
    # 결과창에 띄울 점수들
    Academic_score = 0
    Social_score = 0
    Achievements=[] # 획득한 업적
    Bonus = 0 # 업적보너스점수
    
    
    
    # 하루가 2초이며 3/2일 부터 6/20일까지의 시간을 그린다.
    # 따라서 시간은 111일로 222초가 플레이타임이다.
    time = 0 # 전체 시간
    frame_counter = 0 # 프레임 카운터 60 frames = 1 second  // 가장 처음 0이 안되게 1부터시작
    day = 0 # 하루 = 2초
    music_flag = True  # 배경음 일시정지 시 사용
    pause_time = 0
    pygame.mixer.music.play(-1)  # -1은 무한반복하라는 뜻/ 배경음 반복
    
    
    main_character = Main_character(Window_Width/2, Window_Height-80)
    # 파이썬은 그룹관리를 할 수 있으므로 객체를 만들어 사용한다
    bombs = pygame.sprite.Group() # 이것과 충돌 시 
    friends = pygame.sprite.Group()
    scores = pygame.sprite.Group()
    professors = pygame.sprite.Group()
    quizzes = pygame.sprite.Group()
    GF = pygame.sprite.Group()
    exGFs = pygame.sprite.Group()
    
    GF_appearance = False # 여자친구 등장count
    GF_time = random.randint(1, 100)  # 1일 부터 100일 사이에 하나를 선택 그 때 연인 등장.
    
    bomb_speed = 1
    min_bomb_speed = 1
    max_bomb_speed = 1
    occur_of_bombs = 1
    occur_prob = 10
    total_bomb = 0
    
    main_X = 0
    main_Y = 0
    
    # 교우관계 - > 퍼센트게이지
    Academic_score = 0
    Social_score = 0
    # 이 게이지에 따라 소셜점수에 대한 획득량이 달라짐 낮으면 소셜점수가 떨어질 수 있음.
    Social_gauge = 50 # 초기 소셜 포인트 게이지 
    
    
    paused = False # 일시정지 bool
    while True:
        pygame.display.update()
        fps_clock.tick(FPS)  # 초당 프레임. ->60
        
        
        # 일시정지 시
        if paused:
            for event in pygame.event.get():
                # 다시 p가 눌리면 일시정지 해제
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_p:
                        paused = not paused
                        pygame.mixer.music.unpause()
                    elif event.key == pygame.K_q:
                        return 'initial_screen'
                # 게임종료버튼 X를 누를 때,
                if event.type == pygame.QUIT:
                    return 'quit'
        else:
            frame_counter += 1 # 프레임 세기  -> 초를 셀때 사용
            default_speed = 3 # 기본속도
            
            # 배경 이미지를 프레임마다 업데이트
            screen.blit(background, [0,0])
           
            # 여자친구 등장
            if day == GF_time and GF_appearance ==False:
                G_F =  GirlFriend(random.randint(70, Window_Width - 70),
                            random.randint(70, Window_Height - 70),
                                5,5)
                GF.add(G_F)
                GF_appearance = True # 등장
            
            
            # 100초 때 한번에 생기는 bomb 갯수 증가!
            occur_of_bombs = 1 + int(time / 100) 
            # 80초마다 최고 속도 업!
            max_bomb_speed = default_speed + int(time/ 80)
            
            # 20분의 1의 확률로 1프레임마다 bomb이 생긴다.
            if random.randint(1, occur_prob*2) == 1:
                for i in range(occur_of_bombs):
                    bombs.add(random_bomb(random.randint(min_bomb_speed, max_bomb_speed)))
                    total_bomb += 1
                # 300분의 1확률로 1프레임마다 friend가 생긴다.
                if random.randint(1, occur_prob*1.5) == 1:
                    # 스피드가 0인 경우 2로 바꿔준다.
                    occur_hspeed = random.randint(-max_bomb_speed, max_bomb_speed)
                    occur_vspeed = random.randint(-max_bomb_speed, max_bomb_speed)
                    if occur_hspeed == 0:
                        occur_hspeed = 2
                    if occur_vspeed == 0:
                        occur_vspeed = 2
                    
                    friend = Friend(random.randint(70, Window_Width - 70),
                            random.randint(70, Window_Height - 70),
                            occur_hspeed,occur_vspeed)
                    friends.add(friend)
                # 400분의 1확률로 1프레임마다 quiz가 생긴다.
                if random.randint(1, occur_prob*2) == 1:
                    # 스피드가 0인 경우 2로 바꿔준다.
                    occur_hspeed = random.randint(-max_bomb_speed, max_bomb_speed)
                    occur_vspeed = random.randint(-max_bomb_speed, max_bomb_speed)
                    if occur_hspeed == 0:
                        occur_hspeed = 2
                    if occur_vspeed == 0:
                        occur_vspeed = 2
                    
                    quiz = Quiz(random.randint(70, Window_Width - 70),
                            random.randint(70, Window_Height - 70),
                            occur_hspeed,occur_vspeed)
                    quizzes.add(quiz)
                # 500분의 1확률로 1프레임마다 결별 아이템이 생긴다.
                if random.randint(1, occur_prob*2.5) == 1 and GF_appearance == True:
                    exgf = exGF(random.randint(70, Window_Width - 70),
                            random.randint(70, Window_Height - 70),
                               3,3)
                    exGFs.add(exgf)
                    
            ## 위치고정 ##
            ### 교수님 출현!!##
            proX = Window_Width/2
            proY = Window_Height * 0.1
            
            ## 60초마다 교수님 등장
            if frame_counter % 3600 == 0:
                music_flag = False
                pause_time = frame_counter
                pygame.mixer.music.pause()  # 배경음 일시정지
                if frame_counter // 3600 == 1:
                    professor = Professor(proX, proY , 0,0)
                    professors.add(professor)
                    appearence_professor.play()
                    spread_scores(scores, proX, proY)
                elif frame_counter // 3600 == 2: # 120초
                    professor = Professor(proX, proY, 0,0)
                    professors.add(professor)
                    appearence_professor.play()
                    spread_scores(scores, proX, proY)
                elif frame_counter // 3600 == 3: # 180초
                    professor = Professor(proX, proY , 0,0)
                    professors.add(professor)
                    appearence_professor.play()
                    spread_scores(scores, proX, proY)
            
            # 등장 후 10 초가 지나면 다시 배경음 재생
            if music_flag == False:
                if pause_time + (600) == frame_counter:
                    appearence_professor.stop()
                    pygame.mixer.music.unpause()
            
            # 1일 = 2초
            day = int(time) // 2
            # 1초마다 SP가 1씩떨어짐.
            # 소셜게이지는 0% ~ 120%의 값을 갖는다.
            if frame_counter % 60 == 0: 
                Social_gauge -= 1
            
            # 게이지 최대 최소값 제한
            if Social_gauge > 120:
                Social_gauge = 120
            elif Social_gauge <0:
                Social_gauge = 0
                
            
            draw_text('AP: {}'.format(Academic_score),base_font,screen,Window_Width*0.1 ,20, YELLOW)
            draw_text('day: {}'.format(day),base_font,screen, Window_Width/2 ,20, WHITE)
            time += 1/60
            draw_text('SG: {}%'.format(Social_gauge),base_font,screen,Window_Width *0.9 , 20, WHITE)
            draw_text('SP: {}'.format(Social_score),base_font,screen,Window_Width *0.9 , 60, WHITE)
            
            # 반복적으로 그려준다. 위치를 조정한다.
            # sprite를 그룹으로 사용시 항상 업데이트 해준다.#############
            bombs.update()
            friends.update()
            professors.update()
            scores.update()
            quizzes.update()
            GF.update()
            exGFs.update()
            
            bombs.draw(screen)
            friends.draw(screen)
            professors.draw(screen)
            scores.draw(screen)
            quizzes.draw(screen)
            GF.draw(screen)
            exGFs.draw(screen)
            
            # 충돌 판정 시 효과와 객체 삭제############################
            bomb = main_character.collide(bombs)
            friend = main_character.collide(friends)
            score = main_character.collide(scores)
            quiz = main_character.collide(quizzes)
            GFriend = main_character.collide(GF)
            EXGF = main_character.collide(exGFs)
            
            # bomb과 주인공이 충돌한 것이 있으면 (sprite가 반환되면)
            if bomb:
                bomb.explosion() # 폭발효과
                Academic_score -= 10
            # friend와 주인공이 충돌한 것이 있으면(sprite가 반환되면)
            elif friend:
                Social_gauge += 10
                Item_sound.play()
                friend.kill()
            # score가 주인공과 충돌한 것이 있으면 점수를 반환하고 소리재생
            elif score:
                Academic_score += score.judge_point()
                score.kill()
            elif quiz:
                Academic_score += random.randrange(10, 31, 10) # 10 20 30 점 중 랜덤
                quiz_sound.play()
                quiz.kill()
            elif GFriend:
                girlfriend_sound.play()
                Achievements.append("연인") #업적획득
                GFriend.kill()
            elif EXGF:
                lightening_sound.play()
                Achievements.append("결별") #업적획득
                EXGF.kill()
            
                
            # 주인공의 위치 업데이트
            screen.blit(main_character.image, main_character.rect)
  
            
            # 주인공의 위치를 방향키로 움직인다. 
            # 방향키를 누르고 있으면 그 쪽 방향으로 프레임마다 5씩 이동한다.
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT]:
                main_X = -5
            elif keys[pygame.K_RIGHT]:
                main_X = 5
            if keys[pygame.K_UP]:
                main_Y = -5
            elif keys[pygame.K_DOWN]:
                main_Y = 5
                
#                # 부드럽지 못한 움직임
#                 if event.type in [pygame.KEYDOWN]:
#                     if event.key == pygame.K_LEFT:
#                         main_X -= 5
#                     if event.key == pygame.K_RIGHT:
#                         main_X += 5
#                     if event.key == pygame.K_UP:
#                         main_Y -= 5
#                     if event.key == pygame.K_DOWN:
#                         main_Y += 5
        
            for event in pygame.event.get():
                # 키보드를 때면 멈춤
                if event.type in [pygame.KEYUP]:
                    if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                        main_X = 0
                    elif event.key == pygame.K_UP or event.key == pygame.K_DOWN :
                        main_Y = 0
                        
                # 일시 정지 버튼을 눌렀을 때
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_p:
                        paused = not paused #반대상태
                        # True이면
                        if paused:
                            # 일시정지시 불투명한 surface 가 화면을 채우도록 한다.
                            pause_surf = pygame.Surface((Window_Width, Window_Height))
                            pause_surf.set_alpha(100)
                            screen.blit(pause_surf, pause_surf.get_rect())
                            pygame.mixer.music.pause()
                            # 일시정지 텍스트를 화면에 띄운다.
                            draw_text('일시정지',
                                     pygame.font.Font('Recipekorea_FONT.ttf', 60),
                                     screen, Window_Width/ 2, Window_Height/2, WHITE)
                
                # close 버튼을 눌렀을 때,
                if event.type == pygame.QUIT: 
                    return 'quit'
                
            # 주인공의 움직임 조정 및 경계처리(맵 밖으로 못나가게)
            main_character.set_pos(main_character.rect.x+main_X,main_character.rect.y+main_Y)
#################################################            
            # 초당 소셜스코어 획득량
            if frame_counter % 60 == 0:
                if Social_gauge >= 100:
                    Social_score += 5
                elif Social_gauge >= 80:
                    Social_score += 3
                elif Social_gauge >= 60:
                    Social_score += 1
                elif Social_gauge >= 40:
                    Social_score -= 0
                elif Social_gauge >= 20:
                    Social_score -= 1
                else:
                    Social_score -=3
                
                    
            if frame_counter == (212*60): # 끝나기 10초전부터
                pygame.mixer.music.fadeout(10000) # 10 초 동안 fade out
            
            # 111일이 지나고
            # 시간이 모두 끝났을 때, 업적계산 후 결과화면으로 가기
            if day >= 1 :
                
                #획득한 업적 계산
                if Social_score >=700:
                    Achievements.append("인맥왕")
                    Bonus += 500
                    if Academic_score <=200:
                        Achievements.append("이번 학기는 YOLO")
                        Bonus += -200
                elif Social_score <= 200:
                    Achievements.append("공기와 친한자")
                    Bonus += -100
                    if Academic_score >= 1000:
                        Achievements.append("학점만이 내 길")
                        Bonus += 1000
                if Academic_score >= 1100:
                    Achievements.append("과탑")
                    Bonus += 500
                elif Academic_score <= 200:
                    Achievements.append("학사경고")
                    Bonus += -200
                if GF == 1:
                    Achievements.append("연인")
                    Bonus += 500
                    if ExGF == 1:
                        Achievements.append("결별")
                        Bonus += -600
        
        
# Academic_score = 0
# Social_score = 0
# Achievements=[] # 획득한 업적
# Bonus = 0 # 업적보너스점수


                return 'result_screen'
            
    return 'initial_screen'

####################################################################
def initial_screen():
    while True:
        # 초기화면 이미지

        screen.blit(initial_image, [0, 0])
    
    
        draw_text("마우스 버튼 또는 'S'키를 눌러 게임을 시작하세요. ", pygame.font.Font('Recipekorea_FONT.ttf', 20),
                  screen, Window_Width / 2, Window_Height * 0.25 , BLACK)
        draw_text("'Q'를 누르면 게임이 종료됩니다. ", pygame.font.Font('Recipekorea_FONT.ttf', 20),
                  screen, Window_Width / 2, Window_Height / 1.4,BLACK)
        draw_text("게임 내에서 일시정지는 'P'를 누르세요.", pygame.font.Font('Recipekorea_FONT.ttf', 20),
                  screen, Window_Width / 2, Window_Height * 0.3,BLACK)
        
        
        pygame.display.update()
    
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    return 'quit'
                elif event.key == pygame.K_s:
                    return 'explain_screen'
            if event.type == pygame.MOUSEBUTTONDOWN:
                return 'explain_screen'
            if event.type == pygame.QUIT:
                return 'quit'
        
    return 'initial_screen'
###########################################################
# 설명창
def explain_screen():
    screen.blit(initial_explain, [0,0])
    
    pygame.display.update()
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                return 'quit'
            elif event.key == pygame.K_s:
                return 'play'
        if event.type == pygame.MOUSEBUTTONDOWN:
            return 'play'
        if event.type == pygame.QUIT:
            return 'quit'
    return 'explain_screen'

###########################################################
# 설명창
def result_screen():
    
    global Academic_score, Social_score, Bonus, Achievements
    
    while True:
        
        screen.blit(background2, [0,0])
        draw_text("'S'키를 눌러 게임을 시작하세요. ", pygame.font.Font('Recipekorea_FONT.ttf', 20),
              screen, Window_Width / 2, Window_Height * 0.9 , BLACK)
        draw_text("'Q'를 누르면 게임이 종료됩니다. ", pygame.font.Font('Recipekorea_FONT.ttf', 20),
              screen, Window_Width / 2, Window_Height * 0.95,BLACK)
        draw_text('AP: {}'.format(Academic_score),base_font,screen, Window_Width *0.15, Window_Height * 0.15 , BB)
        draw_text('SP: {}'.format(Social_score),base_font,screen, Window_Width* 0.15, Window_Height * 0.1 , BB)
        
        draw_text('Final Score',base_font,screen,
                  Window_Width * 0.15, Window_Height * 0.8 , BB)
        draw_text(': {}'.format(Social_score+Academic_score+Bonus),base_font,screen,
                  Window_Width * 0.1, Window_Height * 0.86 , BB)
        
        
        
        draw_text('획득한 업적',base_font,screen, Window_Width *0.15, Window_Height * 0.25 , BLACK)
        i = 0.35
        for c in range(0,len(Achievements)):
            draw_text('{}'.format(Achievements[c]),base_font,screen, Window_Width *0.15, Window_Height * i , BLACK)
            i += 0.05
 
        pygame.display.update()
    
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    return 'quit'
                elif event.key == pygame.K_s:
                    return 'initial_screen'
            if event.type == pygame.QUIT:
                return 'quit'
    
    
    
    return 'initial_screen'
###########################################################

# 메인 시스템 루프
def main_loop():
    command = 'initial_screen'
    while command != 'quit':
        if command == 'initial_screen':
            pygame.mixer.music.play(-1) # 배경음 키기
            command = initial_screen()
        elif command == 'explain_screen':
            command = explain_screen()
        elif command == 'play':
            pygame.mixer.music.stop() # 배경음 끄기
            command = Run_game()
        elif command == 'result_screen':
            pygame.mixer.music.stop() # 배경음 끄기
            command = result_screen()
    
    pygame.quit()
    
###########################################################    
main_loop()
        






pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
